In [9]:
import random
import heapq


class TableGen:
    
    def __init__(self, no_of_people, relationships, no_in_pop):
        self.no_of_people = no_of_people
        self.no_in_pop = no_in_pop
        self.table = self.create_first_pop()
        self.relationships = relationships
        self.no_of_relationships = self.generate_relationships()
    
    def generate_relationships(self):
        rels = []
        for i in range(self.relationships):
            a = random.randint(0, self.no_of_people)
            b = random.randint(0, self.no_of_people)
            while a == b:
                b = random.randint(0, self.no_of_people)
            rels.append([a, b])
        for pair in rels:
            pair = sorted(pair)
        return rels
    
    def create_table_set(self):
        people = [-1 for i in range(self.no_of_people)]
        for i in range(self.no_of_people):
            random_seat = random.randint(0, self.no_of_people-1)
            random_place = random.randint(0, self.no_of_people-1)
            while random_seat in people:
                random_seat = random.randint(0, self.no_of_people-1)
            while people[random_place] != -1:
                random_place = random.randint(0, self.no_of_people-1)
            people[random_place] = random_seat
        return people

    def create_first_pop(self):
        pop = []
        for i in range(self.no_in_pop):
            pop.append(self.create_table_set())
        return pop

    
class GeneticAlgorithm:
    
    def __init__(self, no_in_pop, no_of_people, starting_table, relationships, mutation_rate, generations):
        self.best_score = []
        self.best_seating = []
        self.table = starting_table
        self.relationships = relationships
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.no_in_pop = no_in_pop
        
    def calculate_relationship_score(self):
        pop_scores = []
        for seating in self.table:
            score = 0
            for index, value in enumerate(seating):
                if index != 0 and index != len(seating)-1:
                    if sorted([seating[index], seating[index-1]]) in self.relationships:
                        score += 1
                    if sorted([seating[index], seating[index+1]]) in self.relationships:
                        score += 1
                elif index == 0:
                    if sorted([seating[index], seating[index+1]]) in self.relationships:
                        score += 1
                    if sorted([seating[index], seating[len(seating)-1]]) in self.relationships:
                        score += 1
                else:
                    if sorted([seating[index], seating[index-1]]) in self.relationships:
                        score += 1
                    if sorted([seating[index], seating[0]]) in self.relationships:
                        score += 1
            pop_scores.append(score)
        self.best_score.append(max(pop_scores))
        return pop_scores


    def choose_best_indexes_from_pop(self):
        pop_scores = self.calculate_relationship_score()
        max_scores = heapq.nlargest(2, set(pop_scores))
        max_indexes = [i for i, v in enumerate(pop_scores) if v == max_scores[0]]
        self.best_seating.append(self.table[max_indexes[0]])
        if len(max_indexes) >= 2:
            max_indexes = max_indexes[:2]
        elif len(max_indexes) == 1:
            second_best_index = [i for i, v in enumerate(pop_scores) if v == max_scores[1]][0]
            max_indexes = [max_indexes[0], second_best_index]
        max_indexes = [int(i) for i in max_indexes]
        return max_indexes

    @staticmethod
    def generate_child(parent1, parent2):
        child, child_p1, child_p2 = [], [], []
        rand_gene_1 = int(random.random() * len(parent1))
        rand_gene_2 = int(random.random() * len(parent1))
        start_gene = min(rand_gene_1, rand_gene_2)
        end_gene = max(rand_gene_1, rand_gene_2)
        for i in range(start_gene, end_gene):
            child_p1.append(parent1[i])   
        child_p2 = [item for item in parent2 if item not in child_p1]
        child = child_p1 + child_p2
        return child

    def generate_population_of_children(self, parent1, parent2):
        children = []
        for i in range(self.no_in_pop):
            children.append(self.generate_child(parent1, parent2))
        return children

    def mutate(self, child):
        for index, value in enumerate(child):
            if random.random() > self.mutation_rate:
                ind_to_change = int(random.random() * len(child))
                child[index], child[ind_to_change] = child[ind_to_change], child[index]
        return child

    def mutate_population(self, children):
        mutated = []
        for i in children:
            mutated_child = self.mutate(i)
            mutated.append(mutated_child)
        self.table = mutated
        return mutated
        
    def run_genetic_algorithm(self):
        for i in range(self.generations):
            selection = self.choose_best_indexes_from_pop()
            parent1, parent2 = self.table[selection[0]], self.table[selection[1]]
            children = self.generate_population_of_children(parent1, parent2)
            mutated_children = self.mutate_population(children)
    

In [24]:
table = TableGen(100, 200, 100)
ga = GeneticAlgorithm(table.no_in_pop, table.no_of_people, table.table, table.no_of_relationships, 0.92, 100)
ga.run_genetic_algorithm()
print(ga.best_score)
print(ga.best_seating[-1])

[12, 18, 18, 20, 24, 26, 28, 28, 28, 30, 28, 28, 28, 30, 28, 32, 28, 34, 36, 38, 38, 40, 40, 42, 40, 38, 38, 40, 40, 40, 36, 36, 38, 38, 36, 34, 34, 36, 34, 34, 36, 36, 40, 38, 36, 36, 36, 36, 36, 40, 38, 36, 40, 38, 40, 44, 40, 40, 44, 42, 38, 38, 40, 40, 40, 38, 38, 38, 38, 36, 36, 34, 34, 34, 32, 32, 34, 34, 32, 34, 36, 34, 38, 36, 36, 34, 32, 32, 32, 36, 38, 38, 38, 40, 42, 40, 40, 40, 42, 42]
[29, 94, 80, 35, 99, 12, 72, 87, 59, 66, 64, 18, 19, 96, 63, 30, 21, 40, 83, 10, 17, 45, 67, 15, 23, 44, 42, 58, 61, 6, 20, 33, 60, 68, 84, 2, 1, 86, 53, 37, 62, 81, 39, 98, 41, 88, 79, 0, 28, 77, 55, 4, 82, 76, 34, 11, 49, 32, 50, 95, 52, 31, 26, 85, 36, 89, 97, 51, 65, 38, 14, 46, 70, 74, 7, 71, 78, 9, 22, 3, 56, 24, 16, 5, 25, 90, 92, 73, 13, 57, 54, 43, 75, 47, 8, 91, 27, 48, 93, 69]


In [258]:
print(ga.best_score, ga.best_seating)
print(table.no_of_relationships)

[6] [[9, 8, 3, 1, 6, 7, 5, 4, 2, 0]]
[[8, 0], [9, 6], [3, 2], [5, 7], [1, 0], [4, 1], [7, 0], [3, 8], [5, 10], [5, 8], [10, 9], [10, 5], [7, 2], [6, 4], [1, 0], [6, 3], [0, 6], [1, 6], [6, 10], [9, 7]]


In [ ]:
m=[0,0,0,0,0,0,0,0]
max_indexes = [i for i, v in enumerate(pop_scores) if v == m[0]]

In [45]:
x = create_table_set(10)
print()
calculate_relationship_score(x, table.guest_relationships)

0

In [8]:
import random
table = list(range(10))
rels = []
for i in range(5):
    a = random.randint(0,5)
    b = random.randint(0,5)
    while a == b:
        b = random.randint(0,5)
    rels.append([a, b])

In [67]:
import heapq
el=[1,1,1,1]
z=heapq.nlargest(2, set(el))
if len(z) == 1:
    z = z+z
zb

[1, 1]

In [194]:
x=3
[i for i in range(1, x) + range(x+1, 6)]

TypeError: unsupported operand type(s) for +: 'range' and 'range'

In [8]:
x=[1,2,3]
for i, v in enumerate(x):
    if i == 1:
        print(x[i])
        print(x[i-1])
        print(x[i+1])
        print(len(x)-1)

2
1
3
2


In [171]:
[-1 for i in range(len([4,2,2,1]))]

[-1, -1, -1, -1]

In [129]:
z = sorted([1,4,2])
z

[1, 2, 4]

In [49]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    print(geneA, geneB)
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child


breed([0,1,2,3,4,5,6,7,8],[8,7,6,5,4,3,2,1,0])

5 0


[0, 1, 2, 3, 4, 8, 7, 6, 5]

In [19]:
import geneticalgorithm as ga